In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly
import plotly.express as px
import plotly.graph_objs as go
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [2]:
def read_user_data(userno):
    filename=f"data/User{userno}/User{userno}_behacom.csv"
    print(f'[INFO] reading file <{filename}>...')

    data_user=pd.read_csv(filename, encoding='latin-1',chunksize=1000)

    a = next(data_user)
    dtypes_col = a.dtypes.index
    dtypes_type = [i.name for i in a.dtypes.values]
    column_types = dict(zip(dtypes_col, dtypes_type))

    for k,v in column_types.items():
        if k == 'timestamp':
            column_types[k] = 'float64'
        elif ('average' in k):
            column_types[k] = 'float32'
        elif ('stddev' in k):
            column_types[k] = 'float32'
        elif v == 'float64':
            column_types[k] = 'float32'
        elif v == 'int64':
            if (k.startswith('press') or ('counter' in k) or ('usage' in k)):
                column_types[k] = 'int8'
            else:
                column_types[k] = 'int32'

    data_user=pd.read_csv(filename, encoding='latin-1',dtype=column_types)
    #print(data_user.shape)
    return data_user

In [3]:
total_users = 12
df_users = []
basic_info = { 'name':[], 'length':[]}
for i in range(total_users):
    df = read_user_data(i)
    df_users.append(df)
    basic_info['name'].append(f'User {i}')
    basic_info['length'].append(df.shape[0])

[INFO] reading file <data/User0/User0_behacom.csv>...
[INFO] reading file <data/User1/User1_behacom.csv>...
[INFO] reading file <data/User2/User2_behacom.csv>...
[INFO] reading file <data/User3/User3_behacom.csv>...
[INFO] reading file <data/User4/User4_behacom.csv>...
[INFO] reading file <data/User5/User5_behacom.csv>...
[INFO] reading file <data/User6/User6_behacom.csv>...
[INFO] reading file <data/User7/User7_behacom.csv>...
[INFO] reading file <data/User8/User8_behacom.csv>...
[INFO] reading file <data/User9/User9_behacom.csv>...
[INFO] reading file <data/User10/User10_behacom.csv>...
[INFO] reading file <data/User11/User11_behacom.csv>...


In [4]:
basic_info = pd.DataFrame(basic_info)
fig = px.bar(basic_info, x='name', y='length', color='name', title='Distribution of data per user')
fig.show()

In [5]:
# convert timestamp to datetime and add add as column
for i in range(total_users):
    df_users[i]['date_time'] = df_users[i][:]['timestamp'].astype('datetime64[ms]')

In [6]:
fig = go.Figure()
for i in range(total_users):
    df_grp_keystroke = df_users[i].groupby(pd.Grouper(key='date_time',freq='D')).agg({"keystroke_counter": "sum"}).reset_index().sort_values(by='date_time')
    fig.add_trace(go.Scatter(x=df_grp_keystroke['date_time'], y=df_grp_keystroke['keystroke_counter'], name=f'User {i}'))

fig.update_layout(
    title='Keystroke trend per user'
)
fig.show()

In [7]:
fig = go.Figure()
for i in range(total_users):
    df_grp_input_trend = df_users[i].groupby(pd.Grouper(key='date_time',freq='D')).size().to_frame(name='counts').reset_index().sort_values(by='date_time')
    fig.add_trace(go.Scatter(x=df_grp_input_trend['date_time'], y=df_grp_input_trend['counts'], name=f'User {i}'))

fig.update_layout(
    title='Input distribution over time per user'
)
fig.show()

In [8]:
df_user0 = df_users[0].sort_values(by='date_time')
df_user0[:5]

,timestamp,keystroke_counter,erase_keys_counter,erase_keys_percentage,press_press_average_interval,press_press_stddev_interval,press_release_average_interval,press_release_stddev_interval,word_counter,word_average_length,...,system_average_cpu,system_stddev_cpu,current_app_average_mem,current_app_stddev_mem,system_average_mem,system_stddev_mem,received_bytes,sent_bytes,USER,date_time
0,1.574257e+12,14,2,0.14,200.690002,165.470001,107.000000,36.450001,2,5.0,...,13.03,14.42,11.07,0.00,1.150000,2.57,1618568.0,274869.0,0,2019-11-20 13:44:15.128
1,1.574258e+12,37,3,0.08,825.690002,1842.589966,132.679993,96.690002,2,10.5,...,17.50,6.45,1.44,0.08,12.710000,0.97,14835273.0,449913.0,0,2019-11-20 13:45:18.073
2,1.574258e+12,24,0,0.00,2399.870117,5181.549805,122.220001,28.139999,3,5.0,...,19.73,4.89,11.66,0.56,16.500000,6.15,5097547.0,272460.0,0,2019-11-20 13:46:18.881
3,1.574258e+12,28,0,0.00,2087.780029,4272.740234,308.109985,272.179993,1,3.0,...,13.60,3.18,12.55,1.31,25.070000,2.55,3926179.0,293861.0,0,2019-11-20 13:47:19.398
4,1.574258e+12,71,6,0.08,421.290009,1260.380005,153.820007,138.589996,5,11.2,...,16.09,8.56,12.89,1.32,25.719999,2.54,2106149.0,241120.0,0,2019-11-20 13:48:20.304


In [23]:
df_user0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6059 entries, 0 to 6058
Columns: 12052 entries, timestamp to date_time
dtypes: datetime64[ns](1), float32(6022), float64(1), int32(138), int8(5888), object(2)
memory usage: 176.8+ MB


In [36]:
df_grp_user0 = df_user0.groupby(pd.Grouper(key='date_time',freq='15s')).agg(['sum']).reset_index()

df_grp_user0['user_activity'] = df_grp_user0['keystroke_counter'] + df_grp_user0['mouse_average_movement_duration']

In [33]:
df_grp_user0.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 300572 entries, 2019-11-20 13:44:15 to 2020-01-11 18:07:00
Freq: 15S
Columns: 12052 entries, ('timestamp', 'sum') to ('user_active', '')
dtypes: float32(6023), float64(1), int32(138), int8(5888), object(2)
memory usage: 8.6+ GB


In [35]:
df_grp_user0.head()

,timestamp,keystroke_counter,erase_keys_counter,erase_keys_percentage,press_press_average_interval,press_press_stddev_interval,press_release_average_interval,press_release_stddev_interval,word_counter,word_average_length,...,system_average_cpu,system_stddev_cpu,current_app_average_mem,current_app_stddev_mem,system_average_mem,system_stddev_mem,received_bytes,sent_bytes,USER,user_active
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,
date_time,,,,,,,,,,,,,,,,,,,,,
2019-11-20 13:44:15,1.574257e+12,14,2,0.14,200.690002,165.470001,107.000000,36.450001,2,5.0,...,13.03,14.42,11.07,0.00,1.15,2.57,1618568.0,274869.0,0,165.5
2019-11-20 13:44:30,0.000000e+00,0,0,0.00,0.000000,0.000000,0.000000,0.000000,0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0,0.0
2019-11-20 13:44:45,0.000000e+00,0,0,0.00,0.000000,0.000000,0.000000,0.000000,0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0,0.0
2019-11-20 13:45:00,0.000000e+00,0,0,0.00,0.000000,0.000000,0.000000,0.000000,0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0,0.0
2019-11-20 13:45:15,1.574258e+12,37,3,0.08,825.690002,1842.589966,132.679993,96.690002,2,10.5,...,17.50,6.45,1.44,0.08,12.71,0.97,14835273.0,449913.0,0,271.0


In [34]:
fig = px.line(df_grp_user0, x='date_time', y='user_activity', title='User 0 - activity')
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of [('timestamp', 'sum'), ('keystroke_counter', 'sum'), ('erase_keys_counter', 'sum'), ('erase_keys_percentage', 'sum'), ('press_press_average_interval', 'sum'), ('press_press_stddev_interval', 'sum'), ('press_release_average_interval', 'sum'), ('press_release_stddev_interval', 'sum'), ('word_counter', 'sum'), ('word_average_length', 'sum'), ('word_stddev_length', 'sum'), ('word_length_1', 'sum'), ('word_length_2', 'sum'), ('word_length_3', 'sum'), ('word_length_4', 'sum'), ('word_length_5', 'sum'), ('word_length_6', 'sum'), ('word_length_7', 'sum'), ('word_length_8', 'sum'), ('word_length_9', 'sum'), ('word_length_10', 'sum'), ('word_length_11', 'sum'), ('keystrokes_key_0', 'sum'), ('keystrokes_key_1', 'sum'), ('keystrokes_key_2', 'sum'), ('keystrokes_key_3', 'sum'), ('keystrokes_key_4', 'sum'), ('keystrokes_key_5', 'sum'), ('keystrokes_key_6', 'sum'), ('keystrokes_key_7', 'sum'), ('keystrokes_key_8', '